In [26]:
#import packages
import numpy as np
import cv2

In [27]:
#open video
cap = cv2.VideoCapture('/Users/danielrosenthal/Documents/EGR_314/Matt_side3_720.mov')
fps = cap.get(cv2.CAP_PROP_FPS) #get fps
if cap.isOpened() == False:
    print("Cannot open video file.")

In [28]:
#object detection parameters
object_detector = cv2.createBackgroundSubtractorMOG2(history = 100, varThreshold= 75)

In [29]:
# blob detection
params = cv2.SimpleBlobDetector_Params()
 
#threshold
params.minThreshold = 10;
params.maxThreshold = 200; 
# area parameters
params.filterByArea = True
params.minArea = 50
 
# circular parameters
params.filterByCircularity = True
params.minCircularity = 0.1
#params.maxCircularity = 0.7
 
# convexity parameters
params.filterByConvexity = True
params.minConvexity = 0.01
#params.maxConvexity = 0.7

     
# inertia parameters
params.filterByInertia = True
params.minInertiaRatio = 0.01

#params.maxInertiaRatio = 0.1


 
# Create a detector with the parameters
detector = cv2.SimpleBlobDetector_create(params)

In [30]:
while True:
    ret, frame = cap.read()
    
    #region of interest
    height, width, _ = frame.shape
    #print(height, width)
    roi = frame[100:720, 500:1250]
    
    #object detection
    mask = object_detector.apply(roi)

    #cleaning mask
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        #calculate error, remove small elements
        area = cv2.contourArea(cnt)
        if area > 100:
            #cv2.drawContours(roi, [cnt], -1, (0,255,0), 2)
            #drawing rectangle around detected objects
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(roi, (x, y), (x + w, y + h), (0,255,0),3)
    
    #blob detection
    keypoints = detector.detect(mask)
    blank = np.zeros((1, 1))
    blobs = cv2.drawKeypoints(mask, keypoints, blank, (0, 0, 255),
                          cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    number_of_blobs = len(keypoints)
    text = "Number of Circular Blobs: " + str(len(keypoints))
    cv2.putText(blobs, text, (20, 550),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 100, 255), 2)

    #cv2.imshow("roi", roi)
    #cv2.imshow("Frame",frame)
    #cv2.imshow("Mask", mask)
    cv2.imshow("blobs", blobs)
    
    key = cv2.waitKey(120)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()